# ML - Final Project 
Author: Gaurav Kuwar

### Ensure Terraform is installed

Use homebrew on mac to install

In [63]:
terraform --version

Terraform v1.11.4
on darwin_arm64
(.venv) 


### Configure python env and install

In [64]:
export PROJECT_ROOT=$(pwd)
echo $PROJECT_ROOT

(.venv) 
/Users/gauravk/Documents/Classes/Spring 2025/ML_SYS/proj/mlsysops-cms
(.venv) 


In [65]:
python -m venv .venv
source .venv/bin/activate
pip install -r requirements.txt

(.venv) 
(.venv) 
(.venv) 


### Terraform Configure

Update clouds.yaml `REDACTED_UNIQUE_ID` and `REDACTED_SECRET` before proceeding.

In [66]:
cp config/clouds.yaml iac/tf/kvm/clouds.yaml

(.venv) 


In [67]:
cd iac/tf/kvm/
export PATH=/work/.local/bin:$PATH # unset env variables
ls clouds.yaml # check this exists

(.venv) 
(.venv) 
clouds.yaml
(.venv) 


In [68]:
terraform init

Initializing the backend...
Initializing provider plugins...
- Reusing previous version of terraform-provider-openstack/openstack from the dependency lock file
- Using previously-installed terraform-provider-openstack/openstack v1.51.1

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
(.venv) 


In [69]:
# using my netid because of late submission
export TF_VAR_suffix=project26_gk2657
export TF_VAR_key=id_rsa_chameleon

(.venv) 
(.venv) 
(.venv) 


In [70]:
terraform validate

Success! The configuration is valid.

(.venv) 


In [71]:
terraform plan

data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_subnet_v2.sharednet1_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_8000: Reading...
data.openstack_networking_network_v2.sharednet1: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Read complete after 1s [id=e6eff5db-c96d-4e3d-be49-1ade281f4a7a]
data.openstack_networking_secgroup_v2.allow_http_80: Read complete after 1s [id=5df2db36-79f9-4b9b-b6c9-1ece762d83fb]
data.openstack_networking_secgroup_v2.allow_ssh: Read complete after 1s [id=47860acf-9269-4d26-84b1-363824f7e0a2]
data.openstack_networking_secgroup_v2.allow_9001: Read complete after 1s [id=ad2912c0-caa9-49e0-b4

In [72]:
terraform apply -auto-approve

data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_8000: Reading...
data.openstack_networking_network_v2.sharednet1: Reading...
data.openstack_networking_subnet_v2.sharednet1_subnet: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Read complete after 0s [id=ad2912c0-caa9-49e0-b431-b9b3953b91b3]
data.openstack_networking_secgroup_v2.allow_8080: Read complete after 0s [id=e6eff5db-c96d-4e3d-be49-1ade281f4a7a]
data.openstack_networking_secgroup_v2.allow_http_80: Read complete after 0s [id=5df2db36-79f9-4b9b-b6c9-1ece762d83fb]
data.openstack_networking_secgroup_v2.allow_ssh: Read complete after 0s [id=47860acf-9269-4d26-84

In [91]:
cd "$PROJECT_ROOT"

(.venv) 


### Ansible

In [92]:
export ANSIBLE_CONFIG=ansible.cfg
export ANSIBLE_ROLES_PATH=roles

(.venv) 
(.venv) 


In [93]:
ssh-add ~/.ssh/id_rsa_chameleon # add key to agent
# ssh-add -l # then check

Identity added: /Users/gauravk/.ssh/id_rsa_chameleon (gauravk@Gauravs-MacBook-Pro.local)
(.venv) 
(.venv) 


Update `ansible.cfg` with floating IP and then copy

In [94]:
cp config/ansible.cfg iac/ansible/ansible.cfg
cd iac/ansible # change to ansible dir

(.venv) 
(.venv) 


In [95]:
ansible -i inventory.yaml all -m ping

node3 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
node1 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
node2 | SUCCESS => {
    "changed": false,
    "ping": "pong"
}
(.venv) 


In [16]:
ansible-playbook -i inventory.yaml general/hello_host.yml # test it works


PLAY [Hello, world - use Ansible to run a command on each host] ****************

TASK [Run hostname command] ****************************************************
changed: [node1]
changed: [node2]
changed: [node3]

TASK [Show hostname output] ****************************************************
ok: [node1] => 
  msg: The hostname of node1 is node1-mlops-project26-gk2657
ok: [node2] => 
  msg: The hostname of node2 is node2-mlops-project26-gk2657
ok: [node3] => 
  msg: The hostname of node3 is node3-mlops-project26-gk2657

PLAY RECAP *********************************************************************
node1                      : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
node2                      : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
node3                      : ok=2    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

(.venv) 


### Install and Setup Kubernetes Cluster

In [96]:
ansible-playbook -i inventory.yaml --become --become-user=root k8s/k8s_install.yml


PLAY [Kubernetes Dependencies] *************************************************

TASK [Gathering Facts] *********************************************************
ok: [node3]
ok: [node1]
ok: [node2]

TASK [Updates] *****************************************************************
changed: [node2]
changed: [node3]
changed: [node1]

TASK [Disable SWAP] ************************************************************
changed: [node3]
changed: [node1]
changed: [node2]

TASK [Disable SWAP in fstab] ***************************************************
ok: [node3]
ok: [node2]
ok: [node1]

TASK [Create an empty file for the containerd module] **************************
changed: [node3]
changed: [node2]
changed: [node1]

TASK [Configure modules for containerd] ****************************************
changed: [node3]
changed: [node1]
changed: [node2]

TASK [Create an empty file for K8S sysctl parameters] **************************
changed: [node3]
changed: [node1]
changed: [node2]

TASK [Configure 

In [97]:
ansible-playbook -i inventory.yaml --become --become-user=root k8s/k8s_master.yml


PLAY [master] ******************************************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Initialize the cluster] **************************************************
changed: [node1]

TASK [Save output to log file] *************************************************
changed: [node1]

TASK [Create .kube directory] **************************************************
changed: [node1]

TASK [Copy admin.conf to User's kube config] ***********************************
changed: [node1]

TASK [Apply Flannel] ***********************************************************
changed: [node1]

PLAY RECAP *********************************************************************
node1                      : ok=6    changed=5    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   

(.venv) 


In [98]:
ansible-playbook -i inventory.yaml --become --become-user=root k8s/k8s_worker.yml


PLAY [Configure Join Commands on Master Node] **********************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Retrieve Join Command] ***************************************************
changed: [node1]

TASK [Set Join Command] ********************************************************
ok: [node1]

PLAY [Join Worker Nodes] *******************************************************

TASK [Gathering Facts] *********************************************************
ok: [node3]
ok: [node2]

TASK [Enable TCP port 6443 (On Master) is able to connect from Worker;] ********
ok: [node3]
ok: [node2]

TASK [Join worker to cluster] **************************************************
changed: [node3]
changed: [node2]

PLAY RECAP *********************************************************************
node1                      : ok=3    changed=1    unreachable=0    failed=0    skipped=0    rescued=0    ignored=0   
node2                

In [101]:
ansible-playbook -i inventory.yaml --become --become-user=root argocd/argocd_install.yml


PLAY [Install ArgoCD to Kubernetes] ********************************************

TASK [Gathering Facts] *********************************************************
ok: [node1]

TASK [Create argocd namespace] *************************************************
ok: [node1]

TASK [Install ArgoCD] **********************************************************
changed: [node1]

TASK [Wait for ArgoCD admin secret to exist] ***********************************
changed: [node1]

TASK [Fetch ArgoCD initial admin password] *************************************
ok: [node1]

TASK [Show ArgoCD admin password] **********************************************
ok: [node1] => 
  msg: 'ArgoCD admin password: tw0QzPQwTbOPB0Qg'

TASK [Download ArgoCD CLI binary] **********************************************
ok: [node1]

TASK [Verify ArgoCD CLI installation] ******************************************
changed: [node1]

TASK [Show ArgoCD version] *****************************************************
ok: [node1] => 


#### Access the ArgoCD dashboard

`ssh -L 8888:127.0.0.1:8888 cc@A.B.C.D` on local

`kubectl port-forward svc/argocd-server -n argocd 8888:443` on node1

`https://127.0.0.1:8888/` on browser

#### Access the Argo Workflows dashboard

`ssh -L 2746:127.0.0.1:2746 cc@A.B.C.D` on local

`kubectl -n argo port-forward svc/argo-server 2746:274` on node1

`https://127.0.0.1:2746/` on browser

### Setup Docker Hub
Ensure docker is installed locally first, and create account if needed

In [ ]:
docker login

### Build Image

### Push Image

### Deploy Flask App

In [ ]:
ansible-galaxy collection install kubernetes.core

Starting galaxy collection install process
Nothing to do. All requested collections are already installed. If you want to reinstall them, consider using `--force`.
(.venv) 


In [68]:
ansible-playbook -i inventory.yaml --become --become-user=root deploy_flask.yml


PLAY [Deploy public Flask app to Kubernetes] ***********************************

TASK [Gathering Facts] *********************************************************
fatal: [localhost]: FAILED! => changed=false 
  ansible_facts: {}
  failed_modules:
    ansible.legacy.setup:
      failed: true
      module_stderr: |-
        sudo: a password is required
      module_stdout: ''
      msg: |-
        MODULE FAILURE
        See stdout/stderr for the exact error
      rc: 1
  msg: |-
    The following modules failed to execute: ansible.legacy.setup

PLAY RECAP *********************************************************************
localhost                  : ok=0    changed=0    unreachable=0    failed=1    skipped=0    rescued=0    ignored=0   

(.venv) 


: 2

In [62]:
cd "$PROJECT_ROOT"

(.venv) 


#### Delete Infra

In [60]:
cd iac/tf/kvm/

(.venv) 


In [61]:
terraform destroy -auto-approve

data.openstack_networking_secgroup_v2.allow_ssh: Reading...
data.openstack_networking_secgroup_v2.allow_8081: Reading...
data.openstack_networking_subnet_v2.sharednet1_subnet: Reading...
data.openstack_networking_network_v2.sharednet1: Reading...
data.openstack_networking_secgroup_v2.allow_8080: Reading...
data.openstack_networking_secgroup_v2.allow_9090: Reading...
data.openstack_networking_secgroup_v2.allow_8000: Reading...
data.openstack_networking_secgroup_v2.allow_9001: Reading...
data.openstack_networking_secgroup_v2.allow_http_80: Reading...
openstack_networking_network_v2.private_net: Refreshing state... [id=a2d1e7e1-be46-47b1-855b-80babef1355c]
data.openstack_networking_secgroup_v2.allow_9001: Read complete after 1s [id=ad2912c0-caa9-49e0-b431-b9b3953b91b3]
data.openstack_networking_secgroup_v2.allow_http_80: Read complete after 1s [id=5df2db36-79f9-4b9b-b6c9-1ece762d83fb]
data.openstack_networking_secgroup_v2.allow_8081: Read complete after 1s [id=9806d71f-f0bd-4c5d-8c48-f32d